In [1]:
import tensorflow as tf        #设置GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

tf.compat.v1.disable_eager_execution()
hello=tf.constant('Hello,TensorFlow')
config=tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
sess=tf.compat.v1.Session(config=config)
print(sess.run(hello))

b'Hello,TensorFlow'


In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from tqdm import tqdm
import time
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [3]:
# coding:utf-8
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np

from gensim.models import Word2Vec
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers import *
import tensorflow as tf
import warnings

warnings.filterwarnings('ignore')

# 读取数据，简单处理list数据
train = pd.read_csv('G:\\Browser\\GAIDC_preliminary_A\\train.csv')# .sample(frac=0.1)
test = pd.read_csv('G:\\Browser\\GAIDC_preliminary_A\\preliminary_A.csv')# .sample(frac=0.1)

In [4]:
data = pd.concat([train,test])
feature_list = ['transformers', "date_id", 'month', 'is_weekend', 'time', 'L1', 'L2', 'L3', 'L4', 'L5', 'L6']
data['tagid'] = list(data[feature_list].astype(str).values) # 作为单词str

In [5]:
data['tagid'] = data['tagid'].apply(lambda x : x.tolist())
data.iloc[0]['tagid'],type(data.iloc[0]['tagid'])

(['M1',
  'D00003',
  '4',
  '0',
  '0:00',
  '16.4772',
  '11.6558',
  '4.432',
  '6.4635',
  '4.3713',
  '1.1885'],
 list)

In [6]:
# 超参数
# embed_size  embedding size
# MAX_NB_WORDS  tagid中的单词出现次数
# MAX_SEQUENCE_LENGTH  输入tagid list的长度
embed_size = 64
MAX_NB_WORDS = 230637
MAX_SEQUENCE_LENGTH = 1280
# 训练word2vec，这里可以考虑elmo，bert等预训练
w2v_model = Word2Vec(sentences=data['tagid'].tolist(), vector_size=embed_size, window=5, min_count=1,epochs=10)
# 这里是划分训练集和测试数据
X_train = data[:train.shape[0]]['tagid']
X_test = data[train.shape[0]:]['tagid']


In [7]:
# 创建词典，利用了tf.keras的API，其实就是编码一下，具体可以看看API的使用方法
tokenizer = text.Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(list(X_train) + list(X_test))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_train = sequence.pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)
X_test = sequence.pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
word_index = tokenizer.word_index
# 计算一共出现了多少个单词，其实MAX_NB_WORDS我直接就用了这个数据

nb_words = len(word_index) + 1
print('Total %s word vectors.' % nb_words)
# 构建一个embedding的矩阵，之后输入到模型使用
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    try:
        embedding_vector = w2v_model.wv.get_vector(word)
    except KeyError:
        continue
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

y_categorical = train['y'].values



Total 64990 word vectors.


In [9]:
X_train.shape, y_categorical.shape,(MAX_SEQUENCE_LENGTH,)

((38400, 1280), (38400,), (1280,))

In [8]:
def my_model():
    '''构建神经网络多层模型'''
    embedding_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    # 词嵌入（使用预训练的词向量）
    embedder = Embedding(nb_words,
                         embed_size,
                         input_length=MAX_SEQUENCE_LENGTH,
                         weights=[embedding_matrix],
                         trainable=False
                         )
    embed = embedder(embedding_input)
    l = LSTM(128)(embed)
    # flat = BatchNormalization()(l)
    drop = Dropout(0.2)(l)
    # 全连接层，输出时可选择激活函数, activation='sigmoid'
    main_output = Dense(1)(drop) # , activation="sigmoid" 
    model = Model(inputs=embedding_input, outputs=main_output)
    # 可选一个loss损失函数供训练、多个metrics供观察
    model.compile(loss = "mean_absolute_error", optimizer='adam', metrics=['mae','mse'])
    # tf.keras.losses.MAE() tf.keras.losses.BinaryCrossentropy() adam sgd
    return model

In [9]:
# 五折交叉验证
folds = KFold(n_splits=5, shuffle=True, random_state=2019)
oof = np.zeros([len(train), 1])
predictions = np.zeros([len(test), 1])

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train['y'])):
    # KFold五折交叉
    print(f'\nFold_{fold_+1} Featuring ==========={time.asctime(time.localtime(time.time()))}\n')

    model = my_model()
    if fold_ == 0:
        model.summary()

    early_stopping = EarlyStopping(monitor='val_mae', patience=5)
    bst_model_path = "./{}.h5".format(fold_)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

    X_tra, X_val = X_train[trn_idx], X_train[val_idx]
    y_tra, y_val = y_categorical[trn_idx], y_categorical[val_idx]

    model.fit(X_tra, y_tra,
              validation_data=(X_val, y_val),
              epochs=32, batch_size=512, shuffle=True,
              callbacks=[early_stopping, model_checkpoint])

    model.load_weights(bst_model_path)

    oof[val_idx] = model.predict(X_val)

    predictions += model.predict(X_test) / folds.n_splits
    print(predictions)
    del model



Fold_1 Featuring ===========Sat Jan 21 08:07:28 2023

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1280)]            0         
                                                                 
 embedding (Embedding)       (None, 1280, 64)          4159360   
                                                                 
 lstm (LSTM)                 (None, 128)               98816     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 4,258,305
Trainable params: 98,945
Non-trainable params: 4,159,360
_________________________________________________________

In [10]:

train['predict'] = oof

bst_f1_tmp = mean_absolute_error(train['y'].values, train['predict'].values)
print(bst_f1_tmp)

submit = test[['id']]
submit['y'] = predictions
submit.columns = ['user_id', 'y']

submit[['user_id', 'y']].to_csv('open_{}.csv'.format(str(bst_f1_tmp).split('.')[1]), index=False)

4.555567917574724


In [13]:
predictions.mean()

18.815917015292992